# [HW2] Topic Modeling
1. Crawling News
2. Preprocessing
3. Build Term-Document Matrix
4. Topic modeling
5. Visualization

```
🔥 이번 시간에는 Topic Modeling를 직접 크롤링한 뉴스 데이터에 대해서 수행해보는 시간을 갖겠습니다. 

먼저 네이버에서 뉴스 기사를 간단하게 크롤링합니다.
기본적인 전처리 이후 Term-document Matrix를 만들고 이를 non-negative factorization을 이용해 행렬 분해를 하여 Topic modeling을 수행합니다.

t-distributed stochastic neighbor embedding(T-SNE) 기법을 이용해 Topic별 시각화를 진행합니다.
```

In [55]:
!pip install newspaper3k;
!pip install konlpy;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
# 크롤링에 필요한 패키지 설치
from bs4 import BeautifulSoup
from newspaper import Article
from time import sleep
from time import time
from dateutil.relativedelta import relativedelta
from datetime import datetime
from multiprocessing import Pool
import json
import requests
import re
import sys

```
💡 Crawling(크롤링)이란?

크롤링은 웹 페이지에서 필요한 데이터를 추출해내는 작업을 말합니다.
이번 시간에는 정적 페이지인 네이버의 뉴스 신문 기사 웹페이지를 크롤링합니다.

HTML은 설명되어 있는 자료가 많기 때문에 생략하도록 하겠습니다.
HTML 구조 파악 및 태그에 대한 설명은 아래 참고자료를 살펴봐주세요 !
```

참고: [위키피디아: 정적페이지](https://ko.wikipedia.org/wiki/%EC%A0%95%EC%A0%81_%EC%9B%B9_%ED%8E%98%EC%9D%B4%EC%A7%80)

참고: [생활코딩: HTML](https://opentutorials.org/course/2039)

In [57]:
def crawl_news(query: str=None, crawl_num: int=1000, workers: int=4):
    '''뉴스 기사 텍스트가 담긴 list를 반환합니다.

    Keyword arguments:
    query -- 검색어 (default None)
    crawl_num -- 수집할 뉴스 기사의 개수 (defualt 1000)
    workers -- multi-processing시 사용할 thread의 개수 (default 4)
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
    articleList = []
    crawled_url = set()
    keyboard_interrupt = False
    t = time()
    idx = 0
    page = 1

    
    # 서버에 url 요청의 결과를 선언
    res = requests.get(url.format(query))
    print(res)
    sleep(0.5)
    # res를 parsing할 parser를 선언
    bs = BeautifulSoup(res.text, 'html.parser') 
    #응답받은 HTML 내용을 BeautifulSoup 클래스의 객체 형태로 생성/반환합니다.
    #BeautifulSoup 객체를 통해 HTML 코드를 파싱하기 위한 여러 가지 기능을 사용할 수 있습니다.
    #res.text는 응답 받은 내용(HTML)을 Unicode 형태로 반환합니다.

    with Pool(workers) as p:
        while idx < crawl_num:  #페이지를 넘기면 1loop 돈거다. 한페이지에 여러 기사가 있다.          
            table = bs.find('ul', {'class': 'list_news'}) #ul class list_news . 순서없는 리스트로 기사가 여러개 있다.
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')}) #id로 구분 .줄바꿈 외 모든, *0번이상 테이블에 있는 모든 기사이다.
            area_list = [li.find('div', {'class':'news_area'}) for li in li_list] #뉴스거리 중에서 하나 뽑아서 그중에 news_wrap api_ani_sent 를 클래스로 가지는 div 안으로 접근
            a_list = [area.find('a', {'class':'news_tit'}) for area in area_list] #a태그 찾고 저장.

            # 즉 각 뉴스거리중에서 a태그만 a_list에 저장한다.
            
            for n in a_list[:min( len(a_list), crawl_num-idx )]: #한 페이지당 미리 정해둔 크기보다 더 기사가 많으면 짜른다.
                articleList.append(n.get('title')) #a태그의 title을 가져온다. articleList에 저장.
                idx += 1#기사 하나당 idx 하나 카운트.                           
            page += 1                                #페이지 넘기기 위한 변수

            pages = bs.find('div', {'class': 'sc_page_inner'}) #다음페이지로 넘기기 위한 div태그
            next_page_url = [p for p in pages.find_all('a') if p.text == str(page)   ][0].get('href') #찾고자 하는 page넘버와 같은 숫자를 찾고 이것의 하이퍼링크를 저장합니다. 

            req = requests.get('https://search.naver.com/search.naver' + next_page_url) 
            bs = BeautifulSoup(req.text, 'html.parser')
            sleep(0.5)#혹시 문제될까 싶어서 개인적으로 추가함.
    return articleList

```
🔥 이제 '구글'이라는 이름으로 뉴스 기사 1000개의 제목을 크롤링하겠습니다.
```

In [58]:
query = '코로나'

articleList = crawl_news(query)

<Response [200]>


In [59]:
articleList[:10]

['결혼 꺼리는데 코로나까지…신혼부부 1년 새 8.2만쌍 줄어',
 '코로나19 2만5667명 신규 확진… 월요일 13주 만에 최다',
 '서울광장 스케이트장 21일 개장 “코로나 이후 3년만”',
 "경기도 느타리버섯, 코로나로 중단된 홍콩 수출시장 '재도전'",
 '"객실 가동률 코로나 이전 근접"…제주항공, 홍대호텔 본궤도 진입',
 '코로나 견뎠더니 경기 침체…자영업 매출 작년보다 줄었다',
 '"K-컬처 활용해 2024년까지 외국인 관광객 코로나 전 수준 회복"',
 '영끌에 코로나19까지…신혼부부 1년 만에 8만쌍 줄었다',
 '코로나19 다시 증가세…오늘부터 청소년도 개량백신 접종',
 '작년엔 코로나, 올해는 경기침체…자영업자 40% “폐업 고려”']

In [60]:
print(len(articleList))

1000


```
🔥 태거(tagger)를 이용해 한글 명사와 알파벳만을 추출해서 term-document matrix (tdm)을 만들겠습니다.

태거(tagger)는 tokenization에서 조금 더 자세히 다루도록 하겠습니다.
```

참고: [konlpy: morph analyzer](https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/)

## Preprocessing

In [61]:
from konlpy.tag import Okt
from collections import Counter
import json

# Okt 형태소 분석기 선언
t = Okt() #트위터태그

words_list_ = []
vocab = Counter()
tag_set = set(['Noun', 'Alpha'])
stopwords = set(['글자'])

for i, article in enumerate(articleList): #기사의 제목 하나를 다루겠다. article은 title
    if i % 100 == 0:
        print(i)
    
    # tagger를 이용한 품사 태깅
    words = t.pos(article, norm=True, stem=True) #형태소 분석 후 형태소,품사 튜플 형태로 리스트 안에 들어감.
    '''
    ex)
    >>> print(twitter.pos(u'이것도 되나욬ㅋㅋ', norm=True, stem=True))
    [('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되다', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]
    '''
    ############################ ANSWER HERE ################################
    # TODO: 다음의 조건을 만족하는 단어의 리스트를 완성하세요.
    # 조건 1: 명사와 알파벳 tag를 가진 단어
    # 조건 2: 철자 길이가 2이상인 단어 
    # 조건 3: stopwords에 포함되지 않는 단어
    words = [w for w, t in words if t in tag_set and len(w) > 1 and w not in stopwords]
    '''
    words에서 하나씩 꺼내는데 품사가 noun이나 alpha 안에 속하고 두 글자 이상의
    단어이면서 '글자' 라는 단어를 포함하지 않는 words의 단어인 w를 리스트에 저장한다.
    '''
    #########################################################################        

    vocab.update(words) #words에서 중복된 단어를 세고 딕셔너리형태로 바꾸고 vocab에 저장.
    #Counter({'월드컵': 1, '대표팀': 1, '상금': 1, '쏜다': 1, '정몽규': 1, '기부': 1})

    words_list_.append((words, article)) #하나의 기사에 대한 단어(토큰)들과 그 기사의 제목을을 튜플 형태로 리스트 안에 저장한다.
    
vocab = sorted([w for w, freq in vocab.most_common(10000)])#최고 빈도를 보이는 값부터 10000개 중 단어를 vocab에 저장하고 이를 사전순으로 정렬합니다.

word2id = {w: i for i, w in enumerate(vocab)} #단어마다 인덱스를 부여합니다. 여기서 단어는 유일합니다.
words_list = [] 
for words, article in words_list_:  #words 토큰들 article 그 토큰들이 속한 기사의 제목.
    words = [w for w in words if w in word2id] #하나의 기사에 대한 단어들 중 인덱스로 나타낼 수 있는 단어만 저장한다. 인덱스로 나타낼 수 없으면 드롭
    if len(words) > 5: #한 기사의 토큰수가 10이 넘으면 이거때문에 1000->24개로 기사가 추려짐? 10에서 5으로 줄여보자.
        words_list.append((words, article)) #리스트에 그 토큰들과 기사제목을 튜플로 묶어서 리스트에 저장.
        
del words_list_#전처리가 끝났기 때문에 바이바이

0
100
200
300
400
500
600
700
800
900


In [62]:
len(vocab)

1374

## Build document-term matrix

```
🔥 이제 document-term matrix를 만들어보겠습니다.
document-term matrix는 (문서 개수 x 단어 개수)의 Matrix입니다.
```

참고: [Document-Term Matrix](https://wikidocs.net/24559)

In [63]:
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
cnt = 0
dtm = np.zeros((len(words_list), len(vocab)), dtype=np.float32) #기사갯수(crawl_num : 최대 1000 최소: ) 행 , 단어갯수(최대 10000) 열.

for i, (words, article) in enumerate(words_list): #word_list는 한 아이템이 words와 article로 되어 있으며 article은 기사의 제목을 words는 인덱스로서 표현현 수 있는 단어의 리스트가 있다.
    for word in words: #토큰을 꺼내서
        dtm[i, word2id[word]] += 1 #한 기사에서 반복문으로 꺼낸 단어의 인덱스에 해당하는 장소에 1을 더한다. 즉 빈도수를 구한다.

dtm = TfidfTransformer().fit_transform(dtm) 
print(dtm.shape)

(698, 1374)


```
🔥 document-term matrix를 non-negative factorization(NMF)을 이용해 행렬 분해를 해보겠습니다.

💡 Non-negative Factorization이란?

NMF는 주어진 행렬 non-negative matrix X를 non-negative matrix W와 H로 행렬 분해하는 알고리즘입니다.
이어지는 코드를 통해 W와 H의 의미에 대해 파악해봅시다.
```
참고: [Non-negative Matrix Factorization](https://angeloyeo.github.io/2020/10/15/NMF.html)

## Topic modeling

In [64]:
# Non-negative Matrix Factorization
from sklearn.decomposition import NMF

K=5 #5 features
nmf = NMF(n_components=K, alpha=0.1)

```
🔥 sklearn의 NMF를 이용해 W와 H matrix를 구해봅시다.
W는 document length x K, H는 K x term length의 차원을 갖고 있습니다.
W의 하나의 row는 각각의 feature에 얼만큼의 가중치를 줄 지에 대한 weight입니다.
H의 하나의 row는 하나의 feature를 나타냅니다.

우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [65]:
W = nmf.fit_transform(dtm)
H = nmf.components_

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1422: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [66]:
print(W.shape, H.shape) # 5 topics 91 articles 1662 words

(698, 5) (5, 1374)


In [67]:
print(W[:10])
print(H)

[[0.00431276 0.01573639 0.01214544 0.00875695 0.00274374]
 [0.01057951 0.00048597 0.07457577 0.0084037  0.00038446]
 [0.00083865 0.0019813  0.         0.12653505 0.00394723]
 [0.00711813 0.01283504 0.0174262  0.02496494 0.00222665]
 [0.00637153 0.00333001 0.00649942 0.06151743 0.00284813]
 [0.30095818 0.03630037 0.         0.         0.        ]
 [0.00424842 0.00859614 0.01694231 0.01338553 0.00183045]
 [0.00230081 0.00054818 0.00781166 0.12418571 0.0013675 ]
 [0.00119907 0.00198514 0.04185264 0.00713814 0.00229438]
 [0.         0.01158778 0.         0.33469818 0.        ]]
[[0.00000000e+00 2.71107339e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 4.69725840e-03]
 [1.37685992e-03 0.00000000e+00 1.42031735e-04 ... 2.20553691e-02
  4.86044802e-03 0.00000000e+00]
 [3.63848062e-03 0.00000000e+00 3.67934987e-03 ... 0.00000000e+00
  1.17634915e-02 4.61977451e-03]
 [5.94239033e-06 0.00000000e+00 4.34751948e-04 ... 0.00000000e+00
  2.62860562e-03 3.99223583e-03]
 [1.45065550e-04 8.2782

```
🔥 우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [68]:
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]: #k번째 토픽에서 가장 오름차순으로 정렬 -> 뒤부터니 큰 것부터 20개의 인덱스만 추출
        print(vocab[index], end=' ') #빈도수가 가장 높은 것부터 해당하는 인덱스를 넣어서 프린트.
    print()

0th topic
접종 백신 청소년 개량 추가 절기 코로나 오늘 가능 승인 이상 FDA 개월 내일 대상 시작 생후 중구 대전 시행 
1th topic
신규 확진 속보 진자 증가 코로나 위중 사망 유행 전주 기준 겨울철 전남 광주 일요일 주말 대비 전보 강원 발생 
2th topic
방역 완화 중국 코로나 우려 치료 비상 제로 위드 감염 유입 확보 위험 꾸러미 중단 제공 확산 급증 속초시 대웅제약 
3th topic
이전 내년 소매시장 부진 성장 상의 전망 예상 회복 코로나 수준 김포 하네다 호텔 제주항공 홍대 가동 노선 객실 불확실 
4th topic
수원시 지정 안심 식당 추가 코로나 외식 걱정 절기 제주 가동 병상 접종 실시 명칭 중구 대전 토론회 아동 수원 


```
🔥 이번에는 W에서 하나의 Topic (W의 n번째 column)에 접근해서 해당 topic에 대해 값이 가장 높은 3개의 뉴스 기사 제목을 출력해보겠습니다.
```

In [69]:
for k in range(K):
    print(f"==={k}th topic===")
    for index in W[:, k].argsort()[::-1][:3]: #각 topic에 따른 가장 weight가 높은 기사들 3개. #argsort는 정렬하는 인덱스의 어레이를 반환함.
        print(words_list[index][1]) #두번째 값이 제목
    print('\n')

===0th topic===
오늘부터 12~17세 청소년도 코로나 개량 백신 접종 가능
동절기 코로나19 2가백신, 청소년도 접종 가능
내일부터 청소년도 '코로나 개량백신' 접종


===1th topic===
[속보] 코로나 신규 확진 6만2734명…위중증 442명·사망 67명
[속보] 코로나19 신규확진 6만2738명… 전주보다 약 1만명 증가
[속보] 코로나19 신규확진 6만2738명…1주 전보다 1만명 증가


===2th topic===
중국, 갑작스러운 방역 완화로 코로나19 확산 우려
中 방역완화에 비상걸린 北…"코로나 우려에 치료제 확보나서"
中 방역 완화에 北 비상..."코로나19 유입 우려·치료제 확보 나서"


===3th topic===
상의 "내년 소매시장 1.8% 성장 전망…코로나 이전보다 부진"
상의 "내년 소매시장 1.8% 성장 예상...코로나 이전보다 부진"
대한상의 “내년 소매시장 1.8% 성장 예상…코로나 이전보다 부진”


===4th topic===
수원시 '코로나19 안심식당' 228곳 추가 지정
수원시, '코로나19 안심식당' 228개 추가 지정
수원시, '코로나19 안심식당' 228개소 추가 지정




```
❓ 2번째 토픽에 대해 가장 높은 가중치를 갖는 제목 5개를 출력해볼까요?
```

In [70]:
#TODO
index_list = W[:, 2].argsort()[::-1][:5] #1th topic , 정렬해서 뒤부터 다섯개 인덱스를를
for index in index_list:  #words_list에 넣어서 프린트
    print(words_list[index][1])

중국, 갑작스러운 방역 완화로 코로나19 확산 우려
中 방역완화에 비상걸린 北…"코로나 우려에 치료제 확보나서"
中 방역 완화에 北 비상..."코로나19 유입 우려·치료제 확보 나서"
中 방역완화에 北 ‘비상’…“코로나 유입 우려, 치료제 확보나서”
中 방역완화에 北 '비상'…"코로나 유입 우려, 치료제 확보나서"


```
🔥 이번에는 t-SNE를 이용해 Topic별 시각화를 진행해보겠습니다.

💡 t-SNE는 무엇인가요?

t-Stochastic Neighbor Embedding(t-SNE)은 고차원의 벡터를 
저차원(2~3차원) 벡터로 데이터간 구조적 특징을 유지하며 축소를 하는 방법 중 하나입니다.

주로 고차원 데이터의 시각화를 위해 사용됩니다.
```

참고: [lovit: t-SNE](https://lovit.github.io/nlp/representation/2018/09/28/tsne/#:~:text=t%2DSNE%20%EB%8A%94%20%EA%B3%A0%EC%B0%A8%EC%9B%90%EC%9D%98,%EC%9D%98%20%EC%A7%80%EB%8F%84%EB%A1%9C%20%ED%91%9C%ED%98%84%ED%95%A9%EB%8B%88%EB%8B%A4.)

참고: [ratsgo: t-SNE](https://ratsgo.github.io/machine%20learning/2017/04/28/tSNE/)

## Visualization

In [71]:
from sklearn.manifold import TSNE

# n_components = 차원 수
tsne = TSNE(n_components=2, init='pca', verbose=1)

# W matrix에 대해 t-sne를 수행합니다.
W2d = tsne.fit_transform(W)

# 각 뉴스 기사 제목마다 가중치가 가장 높은 topic을 저장합니다.
topicIndex = [v.argmax() for v in W]

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 698 samples in 0.001s...
[t-SNE] Computed neighbors for 698 samples in 0.067s...
[t-SNE] Computed conditional probabilities for sample 698 / 698
[t-SNE] Mean sigma: 0.005988


/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] KL divergence after 250 iterations with early exaggeration: 53.486183
[t-SNE] KL divergence after 1000 iterations: 0.256727


In [72]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article for words, article in words_list],
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color'
)

# interaction
p.legend.location = "top_left"
hover = p.select({'type': HoverTool})
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)